<a href="https://colab.research.google.com/github/amanvarma16/Semantic-search-using-se2seq-/blob/master/Text_Summarization_and_Semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q ktext
!pip install -q annoy

spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.


In [4]:
import json
from urllib.request import urlopen

from annoy import AnnoyIndex
from keras import optimizers
from keras.layers import Input, Dense, LSTM, GRU, Embedding, Lambda, BatchNormalization
from keras.models import load_model, Model
from keras import optimizers
from keras.preprocessing.sequence import pad_sequences
from keras.utils import get_file, to_categorical
from ktext.preprocess import processor
import numpy as np
import pandas as pd
import random
from tqdm import tqdm

Using TensorFlow backend.


## Datasets for code snippets and Language Model 

In [5]:
!wget http://www.phontron.com/download/conala-corpus-v1.1.zip
!unzip -o conala-corpus-v1.1.zip

--2019-01-28 05:35:39--  http://www.phontron.com/download/conala-corpus-v1.1.zip
Resolving www.phontron.com (www.phontron.com)... 208.113.196.149
Connecting to www.phontron.com (www.phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52105440 (50M) [application/zip]
Saving to: ‘conala-corpus-v1.1.zip’

conala-corpus-v1.1. 100%[===================>]  49.69M  13.1MB/s    in 4.3s    

2019-01-28 05:35:44 (11.5 MB/s) - ‘conala-corpus-v1.1.zip’ saved [52105440/52105440]

Archive:  conala-corpus-v1.1.zip
   creating: conala-corpus/
  inflating: conala-corpus/conala-mined.jsonl  
  inflating: conala-corpus/conala-train.json  
  inflating: conala-corpus/conala-test.json  


In [0]:
with open('conala-corpus/conala-mined.jsonl' , 'r') as f :
  lines = [json.loads(line) for line in f.readlines()]
  source_docs = [line['snippet'] for line in lines]
  target_docs = [line ['intent'] for line in lines]

In [0]:
with open('conala-corpus/conala-train.json', 'r') as f:
    lines = json.load(f)
train_source_docs = [line['snippet'] for line in lines]
train_target_docs = [line['intent'] for line in lines]
test_docs = [line['rewritten_intent'] for line in lines if line['rewritten_intent']]

In [0]:
with open('conala-corpus/conala-test.json', 'r') as f:
    lines = json.load(f)
test_source_docs = [line['snippet'] for line in lines]
test_target_docs = [line['intent'] for line in lines]

In [0]:
#Subset

source_docs = source_docs[:200000]
target_docs = target_docs[:200000]

Pre-Processing

Cleaning - Lowercase, punctuation, 

Tokenize - Split each document into a list of words

Build Vocab - ids for tokens

Padding - To make the length of the documents uniform




In [10]:
for x in source_docs[:10]:
  print (x)

sorted(l, key=lambda x: (-int(x[1]), x[0]))
[int(x) for x in str(num)]
c.decode('unicode_escape')
parser.add_argument('-t', dest='table', help='', nargs='+')
datetime.datetime.strptime(s, '%Y-%m-%dT%H:%M:%SZ')
np.array(x._data).reshape(x.size[::-1]).T
soup.get_text().replace('\n', '\n\n')
re.sub('(?<!\\S)((\\S+)(?:\\s+\\2))(?:\\s+\\2)+(?!\\S)', '\\1', s)
mylist.sort(key=lambda d: (d['weight'], d['factor']))
itertools.combinations


In [11]:
for x in target_docs[:10]:
  print(x)

Sort a nested list by two elements
converting integer to list in python
Converting byte string in unicode string
List of arguments with argparse
How to convert a Date string to a DateTime object?
How to efficiently convert Matlab engine arrays to numpy ndarray?
Converting html to text with Python
regex for repeating words in a string in Python
Ordering a list of dictionaries in python
Two Combination Lists from One List


In [12]:
proc = processor(hueristic_pct_padding=.7, keep_n=5000)
vecs = proc.fit_transform(target_docs)

 See full histogram by insepecting the `document_length_stats` attribute.


In [13]:
print ('orginal list : ',target_docs[0])

print ('transformed list :', vecs[0])

orginal list :  Sort a nested list by two elements
transformed list : [  0   0   0 135   2 145  11  45  37  62]


In [14]:
proc.token_count_pandas().head()

,count
a,103489
python,96545
in,94008
to,92498
how,70208


In [15]:
vocab_size = max(proc.id2token.keys()) + 1
max_length = proc.padding_maxlen

print('vocab size: ', vocab_size)
print('max length allowed for documents: ', max_length)

vocab size:  5002
max length allowed for documents:  10


In [16]:
sequences = []
for arr in tqdm(vecs):
    non_zero = (arr != 0).argmax()
    for i in range(non_zero, len(arr)):
        sequences.append(arr[:i+1])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

print (sequences[2])

print (X)

print (y)

100%|██████████| 200000/200000 [00:01<00:00, 153647.27it/s]


[  0   0   0   0   0   0   0 135   2 145]
[[   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0  135]
 [   0    0    0 ...    0  135    2]
 ...
 [   0    0    0 ...    0    0   13]
 [   0    0    0 ...    0   13 1330]
 [   0    0    0 ...   13 1330    8]]
[ 135    2  145 ... 1330    8  194]


In [17]:
i = Input(shape = (max_length-1,))
x= Embedding(vocab_size,256,input_length = max_length-1) (i)
x = LSTM(256, return_sequences = True)(x)
last_timestep = Lambda(lambda x : x[:,-1,:]) (x)
last_timestep = Dense(vocab_size, activation ='softmax') (last_timestep)
model = Model(i,last_timestep)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 9)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 9, 256)            1280512   
_________________________________________________________________
lstm_1 (LSTM)                (None, 9, 256)            525312    
_________________________________________________________________
lambda_1 (Lambda)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5002)              1285514   
Total params: 3,091,338
Trainable params: 3,091,338
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, epochs=1, batch_size=500, validation_split=0.1)

Train on 1466839 samples, validate on 162983 samples
Epoch 1/1
1466839/1466839 [==============================] - 142s 97us/step - loss: 5.2690 - acc: 0.1967 - val_loss: 4.8326 - val_acc: 0.2431


Using a cached model


In [19]:
assert len(target_docs) == 200000, 'target_docs should be truncated to the first 200k rows to use the cached model.'

fname = get_file(fname='kdd_lm_v2.h5', origin='https://storage.googleapis.com/kdd-seq2seq-2018/kdd_lm_v2.h5', )
model = load_model(fname)

31850496/31848704 [==============================] - 1s 0us/step


In [0]:
def generate_seq(model, proc, n_words, seed_text):
    in_text = seed_text
    for _ in range(n_words):
        vec = proc.transform([in_text])[:,1:]
        index = np.argmax(model.predict(vec, verbose=0), axis=1)[0]
        out_word = ''
        if index == 1:
            out_word = '_unk_'
        else:
            out_word = proc.id2token[index]
        in_text += ' ' + out_word
    return in_text

In [21]:
generate_seq(model,proc,max_length,'string')

'string substitution performance in python logging and _unk_ in to c'

## We trained our language model to learn representations of sentences in our corpus

In [0]:
embedding_model = Model(inputs=model.inputs, outputs=model.layers[-3].output)

In [23]:
input_sequence = test_docs[random.randint(0, len(test_docs))]
print('input sequence: ', input_sequence, '\n\nhidden states:\n')
vec = proc.transform([input_sequence])[:,1:]
embedding_model.predict(vec)

input sequence:  urlencode a querystring 'string_of_characters_like_these:$#@=?%^Q^$' in python 2 

hidden states:



array([[[ 0.        ,  0.        , -0.        , ...,  0.        ,
         -0.41064757,  0.4782344 ],
        [ 0.3200685 ,  0.7614881 , -0.00804727, ...,  0.5636641 ,
         -0.07097925,  0.90842754],
        [ 0.34599552,  0.9610021 ,  0.761538  , ...,  0.18917751,
          0.69573283,  0.98709613],
        ...,
        [ 0.        ,  0.        , -0.        , ...,  0.        ,
         -0.        ,  0.        ],
        [ 0.04048524,  0.        ,  0.07371317, ...,  0.6745981 ,
         -0.        ,  0.        ],
        [ 0.        ,  0.        , -0.7506168 , ...,  0.        ,
         -0.        ,  0.        ]]], dtype=float32)

In [0]:
test_vecs = proc.transform(test_docs)

In [0]:
hidden_states = embedding_model.predict(test_vecs[:, 1:])

In [0]:
mean_vecs = np.mean(hidden_states, axis=1)
max_vecs = np.max(hidden_states, axis=1)
sum_vecs = np.sum(hidden_states, axis=1)

In [27]:
print (hidden_states[1])

[[-0.          0.32860067 -0.         ...  0.          0.
   0.7615874 ]
 [-0.6193741   0.         -0.         ...  0.         -0.0956229
   0.9640264 ]
 [-0.         -0.         -0.7721795  ...  0.         -0.
   0.9950546 ]
 ...
 [-0.73660165  0.         -0.         ...  0.         -0.18717118
   0.38312688]
 [-0.          0.         -0.25832775 ...  0.         -0.73036677
   0.99999976]
 [-0.          0.         -0.99879473 ...  0.         -0.38547844
   0.99999994]]


Build Vector Indices. Use Annoy

In [28]:
dimension = hidden_states.shape[-1]
index = AnnoyIndex(dimension)
for i, v in enumerate(sum_vecs):
    index.add_item(i, v)
index.build(10)

True

In [29]:
input_sequence = test_docs[random.randint(0, len(test_docs))]
print('Query: ', input_sequence)

vec = proc.transform([input_sequence])[:,1:]
vec = np.sum(embedding_model.predict(vec), axis=1)
ids, _ = index.get_nns_by_vector(vec.T, 10, include_distances=True)

print('\nSearch Results:')
[test_docs[i] for i in ids][1:]

Query:  Convert ascii value 'P' to binary

Search Results:


["convert ascii value 'a' to int",
 'Convert hex string `hexString` to int',
 'convert hex string `s` to decimal',
 'Convert hex string `s` to integer',
 'Convert hex string "deadbeef" to integer',
 'Convert hex string "0xa" to integer',
 'convert hex string "0xff" to decimal',
 'convert hex string "FFFF" to decimal',
 "convert hex string '0xdeadbeef' to decimal"]

## SEQ 2 SEQ MODEL

## We'll use it for a creative task : Given a snippet of code, it should be able to generate a description of that code.



We'll use the simplest of the seq2seq model, an encoder-decoder network using RNNs

In [30]:
print('source (code input): ', source_docs[2])
print('target (description output): ', target_docs[2])


source (code input):  c.decode('unicode_escape')
target (description output):  Converting byte string in unicode string


In [31]:
target_proc = processor(append_indicators=True, hueristic_pct_padding=.7, keep_n=12000, padding ='post')
target_vecs = target_proc.fit_transform(target_docs)

 See full histogram by insepecting the `document_length_stats` attribute.


In [32]:
source_proc = processor(hueristic_pct_padding=.7, keep_n=15000)
source_vecs = source_proc.fit_transform(source_docs)

 See full histogram by insepecting the `document_length_stats` attribute.


In [0]:
encoder_input_data = source_vecs
encoder_seq_len = encoder_input_data.shape[1]

decoder_input_data = target_vecs[:, :-1]
decoder_target_data = target_vecs[:, 1:]

num_encoder_tokens = max(source_proc.id2token.keys()) + 1
num_decoder_tokens = max(target_proc.id2token.keys()) + 1

In [34]:
print (encoder_input_data [2])
print (decoder_input_data [2])

print (decoder_target_data[2])

[  0   0   0   0   0   0  20 259 522 633]
[  2 164 514  17   6 144  17   3   0   0   0   0   0   0]
[164 514  17   6 144  17   3   0   0   0   0   0   0   0]


# Encoder Model

The role of the encoder is to extract features and generate a representation of the input sequence, which in this case is a snippet of code.

In [0]:
word_emb_dim=512
hidden_state_dim=1024
encoder_seq_len=encoder_seq_len
num_encoder_tokens=num_encoder_tokens
num_decoder_tokens=num_decoder_tokens

encoder_inputs = Input(shape=(encoder_seq_len,), name='Encoder-Input')
x = Embedding(num_encoder_tokens, word_emb_dim, name='Body-Word-Embedding', mask_zero=False)(encoder_inputs)
x = BatchNormalization(name='Encoder-Batchnorm-1')(x)
_, state_h = GRU(hidden_state_dim, return_state=True, name='Encoder-Last-GRU', dropout=.5)(x)
encoder_model = Model(inputs=encoder_inputs, outputs=state_h, name='Encoder-Model')
seq2seq_encoder_out = encoder_model(encoder_inputs)

In [36]:
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder-Input (InputLayer)   (None, 10)                0         
_________________________________________________________________
Body-Word-Embedding (Embeddi (None, 10, 512)           7681024   
_________________________________________________________________
Encoder-Batchnorm-1 (BatchNo (None, 10, 512)           2048      
_________________________________________________________________
Encoder-Last-GRU (GRU)       [(None, 1024), (None, 102 4721664   
Total params: 12,404,736
Trainable params: 12,403,712
Non-trainable params: 1,024
_________________________________________________________________


# Decoder Model

In [0]:
decoder_inputs = Input(shape=(None,), name = 'Decoder-Input')
dec_emb = Embedding(num_decoder_tokens, word_emb_dim, name='Decoder-Word-Embedding', mask_zero=False)(decoder_inputs)
dec_bn = BatchNormalization(name='Decoder-Batchnorm-1')(dec_emb)
decoder_gru = GRU(hidden_state_dim, return_state=True, return_sequences=True, name='Decoder-GRU', dropout=.5)
decoder_gru_output, _ = decoder_gru(dec_bn, initial_state=seq2seq_encoder_out)
x = BatchNormalization(name='Decoder-Batchnorm-2')(decoder_gru_output)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Final-Output-Dense')
decoder_outputs = decoder_dense(x)

Sequence to sequence model
We can connect the encoder and decoder together to create the sequence to sequence model.

[ ]

seq2seq_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:

seq2seq_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [39]:
seq2seq_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Decoder-Word-Embedding (Embeddi (None, None, 512)    5846528     Decoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-Input (InputLayer)      (None, 10)           0                                            
__________________________________________________________________________________________________
Decoder-Batchnorm-1 (BatchNorma (None, None, 512)    2048        Decoder-Word-Embedding[0][0]     
__________________________________________________________________________________________________
Encoder-Mo

In [0]:
batch_size = 1024
epochs = 1

seq2seq_model.compile(optimizer=optimizers.Nadam(lr=0.00005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = seq2seq_model.fit([encoder_input_data, decoder_input_data],
                            np.expand_dims(decoder_target_data, -1),
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_split=0.1)

In [0]:
!wget https://storage.googleapis.com/kdd-seq2seq-2018/kdd_seq2seq_weights.h5

In [0]:
seq2seq_model.load_weights('kdd_seq2seq_weights.h5')

### Now Extract Encoder and Decoder from the model layers.
Encoder - To create embeddings for the raw input text

Decoder - To generate descriptions for the code snippets 

In [0]:
def extract_decoder_model(model):
    latent_dim = model.get_layer('Encoder-Model').output_shape[-1]
    decoder_inputs = model.get_layer('Decoder-Input').input
    dec_emb = model.get_layer('Decoder-Word-Embedding')(decoder_inputs)
    dec_bn = model.get_layer('Decoder-Batchnorm-1')(dec_emb)
    gru_inference_state_input = Input(shape=(latent_dim,), name='hidden_state_input')
    gru_out, gru_state_out = model.get_layer('Decoder-GRU')([dec_bn, gru_inference_state_input])
    dec_bn2 = model.get_layer('Decoder-Batchnorm-2')(gru_out)
    dense_out = model.get_layer('Final-Output-Dense')(dec_bn2)
    decoder_model = Model([decoder_inputs, gru_inference_state_input], [dense_out, gru_state_out])
    return decoder_model

### We can extract Encoder as a general purpose feature Extractor

In [43]:
encoder_model = seq2seq_model.get_layer('Encoder-Model')
for layer in encoder_model.layers :
  layer.trainable = False
  
  
decoder_model = extract_decoder_model(seq2seq_model)
decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Decoder-Word-Embedding (Embeddi (None, None, 512)    5846528     Decoder-Input[0][0]              
__________________________________________________________________________________________________
Decoder-Batchnorm-1 (BatchNorma (None, None, 512)    2048        Decoder-Word-Embedding[1][0]     
__________________________________________________________________________________________________
hidden_state_input (InputLayer) (None, 1024)         0                                            
__________________________________________________________________________________________________
Decoder-GR

### Prediction using our decoder model for raw input in test_source

In [57]:
i = random.randint(0,len(test_source_docs))

max_len = target_proc.padding_maxlen
raw_input_text = test_source_docs[i]

raw_tokenized = source_proc.transform([raw_input_text])
encoding = encoder_model.predict(raw_tokenized)
original_encoding = encoding
state_value = np.array(target_proc.token2id['_start_']).reshape(1, 1)

decoded_sentence = []
stop_condition = False
while not stop_condition:
    preds, st = decoder_model.predict([state_value, encoding])
    pred_idx = np.argmax(preds[:, :, 2:]) + 2
    pred_word_str = target_proc.id2token[pred_idx]

    if pred_word_str == '_end_' or len(decoded_sentence) >= max_len:
        stop_condition = True
        break
    decoded_sentence.append(pred_word_str)

    # update the decoder for the next word
    encoding = st
    state_value = np.array(pred_idx).reshape(1, 1)

print('sample code from test set:\n------------------------\n', raw_input_text)
print('\nground truth:\n------------------------\n', test_target_docs[i])
print('\npredicted description:\n------------------------')
print(' '.join(decoded_sentence))

 
 

sample code from test set:
------------------------
 re.findall('n(?<=[^n]n)n+(?=[^n])(?i)', s)

ground truth:
------------------------
 Regex, find pattern only in middle of string

predicted description:
------------------------
dictionary bail arange spesific rich pdf dataframe parenthesis cyrillic startmenu works cryptographic notebooks dependencies potential


## Generate Embeddings

We need two embeddings

Embeddings for the code snippets, from the seq2seq encoder.

Embeddings for the docstrings, from the language model

### Embedding for code snippets

In [0]:
train_source_emb = encoder_model.predict(source_proc.transform(source_docs))

Embeddings for target docs which are docstrings, natural language 

In [0]:
train_target_vecs = proc. transform(target_docs)
hidden_states = embedding_model.predict(train_target_vecs[:,1:])

Summarize hidden states from embedding model

In [0]:
mean_vecs = np.mean(hidden_states,axis=1)
max_vecs = np.max(hidden_states, axis =1)
sum_vecs = np.sum(hidden_states, axis =1)

train_target_emb = mean_vecs

In [63]:
print('source embedding shape on training set: ', train_source_emb.shape)
print('target embedding shape on training set: ', train_target_emb.shape)

source embedding shape on training set:  (200000, 1024)
target embedding shape on training set:  (200000, 200)


# Construct a Joint Vector Space (Semantic Code Search)
Right now we have a way of representing:

a blob of code as a vector using the encoder of the sequence-to-sequence model, and
the code descriptions as a vector using the language model.
However, these two vector spaces are not related to eachother. It can be useful to project the vectors for code and descriptions into the same space so that we can search code with natural language.

In [64]:
#Most of the pieces for this step come from prior steps in this tutorial. In this step, we will fine-tune the seq2seq model to predict docstring embeddings instead of docstrings.


inp = Input(shape=(train_source_emb.shape[1],))
x = Dense(train_target_emb.shape[1], use_bias=False)(inp)
# x = BatchNormalization()(x)
# x = Dense(train_target_emb.shape[1])(x)
modal_model = Model([inp], x)
modal_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               204800    
Total params: 204,800
Trainable params: 204,800
Non-trainable params: 0
_________________________________________________________________


In [65]:
modal_model.compile(optimizer=optimizers.Nadam(lr=0.0001), loss='cosine_proximity', metrics=['accuracy'])

batch_size = 1200
epochs = 20
history = modal_model.fit([train_source_emb], train_target_emb,
                          batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 180000 samples, validate on 20000 samples
Epoch 1/20
180000/180000 [==============================] - 2s 12us/step - loss: -0.5107 - acc: 0.0215 - val_loss: -0.6320 - val_acc: 0.0316
Epoch 2/20
180000/180000 [==============================] - 1s 7us/step - loss: -0.6623 - acc: 0.0384 - val_loss: -0.6692 - val_acc: 0.0448
Epoch 3/20
180000/180000 [==============================] - 1s 7us/step - loss: -0.6866 - acc: 0.0488 - val_loss: -0.6871 - val_acc: 0.0518
Epoch 4/20
180000/180000 [==============================] - 1s 7us/step - loss: -0.7000 - acc: 0.0560 - val_loss: -0.6990 - val_acc: 0.0584
Epoch 5/20
180000/180000 [==============================] - 1s 7us/step - loss: -0.7085 - acc: 0.0599 - val_loss: -0.7059 - val_acc: 0.0615
Epoch 6/20
180000/180000 [==============================] - 1s 7us/step - loss: -0.7138 - acc: 0.0630 - val_loss: -0.7104 - val_acc: 0.0657
Epoch 7/20
180000/180000 [==============================] - 1s 7us/step - loss: -0.7175 - acc: 0.0658 - val_

## APPLICATION - Semantic search

Warning : Results of this portion are minimal and not very good.

1) **Vectorize all of the code** :  We are going to push all the code through the encoder -> then modal_model to project the code into the same vector-space as our phrase embeddings.

In [66]:
vec_code = modal_model.predict(encoder_model.predict(source_proc.transform(source_docs)))
vec_code.shape

(200000, 200)

2)** Place all vectorized code into search index** : We use Spotify's Annoy 

In [68]:
dimension = vec_code.shape[-1]
index = AnnoyIndex(dimension)

for i,v in enumerate(vec_code):
  index.add_item(i,v)
index.build(10)

True

3) **Create function to vectorize queries **: we use embedding model for natural language to create sentence embeddings

In [0]:
def vectorize_queries(txt) :
  vec = proc.transform([txt])[:,1:]
  emb = np.mean(embedding_model.predict(vec),axis=1)
  return emb

**Given a query, lookup the closest code in vector space**. . 

We can wrap everything in one function that:


1)vectorizes a string query

2)lookup the nearest vectorized code (which is in the same vector-space as the string query)

3)retrieve the original code and display it for the user.

In [0]:
def search (inp) :
  inp = vectorize_queries (inp)
  ids, dist = index.get_nns_by_vector (inp.squeeze(),10,include_distances = True)
  
  for i, dist in zip(ids,dist) :
    print(f'dist: {dist:.2f}\n{source_docs[i]}\n')

## **The Demo**

In [71]:
search('read csv file into pandas dataframe')

dist: 0.50
df['Test'] = np.maximum(df['Open'], df['Close'])

dist: 0.50
with open('write.csv', 'a') as f:
    df.to_csv(f, header=False, index=False)

dist: 0.50
data_df = pd.read_csv('data.csv')

dist: 0.50
import pandas as pd
df = pd.DataFrame.from_csv('data.csv')
result = df[(df.Sex == 'female') & (df.Survived == False)]

dist: 0.50
df = pd.DataFrame.from_csv('data.csv')

dist: 0.50
df = pd.read_csv('data.csv')

dist: 0.51
df.to_csv('data.csv')

dist: 0.51
resp = make_response(df.to_csv())

dist: 0.51
titanic = pd.read_csv('titanic_data.csv')

dist: 0.51
df = pandas.read_csv('Yourfile.csv')



In [72]:
search('sort list by descending order')

dist: 0.54
match = re.match('\\d{4}-\\d{2}-\\d{2}T\\d{2}:\\d{2}Z', date_string)

dist: 0.55
d = {}
for t in l:
    d.setdefault(t[0], {})[t[1]] = t[2]

dist: 0.55
"""-""".join(a + b for a, b in zip(t, t))

dist: 0.55
t2 = [(a + b) for a, b in zip(t, t[1:])]
t2.append(t[0] + t[-1])

dist: 0.55
t2 = [(a + b) for a, b in zip(t, t[1:])]

dist: 0.55
def compare(s, t):
    return Counter(s) == Counter(t)

dist: 0.55
d = {'A': 'a', 'B': {'C': 'c', 'D': 'd', 'E': 'e'}}

dist: 0.55
d = {'A': 'a', 'B': {'C': 'c', 'D': 'd', 'E': 'e'}}
with open('result.yml', 'w') as yaml_file:
    yaml.dump(d, yaml_file, default_flow_style=False)

dist: 0.55
d = {'A': 'a', 'B': {'C': 'c', 'D': 'd', 'E': 'e'}}
with open('result.yaml', 'w') as f:
    yaml.dump(d, f, default_flow_style=False)

dist: 0.55
d = {'A': 'a', 'B': {'C': 'c', 'D': 'd', 'E': 'e'}}



In [73]:
search ('pandas dataframe')

dist: 0.86
info = {}

dist: 0.88
super(programmers, self).info()

dist: 0.88
a = 42 * np.ones(16, dtype=np.float32)

dist: 0.89
def __init__(self, a=None, b=None, c=None):
    self.a = a

dist: 0.89
a = np.zeros((3000, 300), dtype=np.float32)

dist: 0.89
np_data = np.zeros((10, 10), dtype=np.float32)

dist: 0.89
self.y = []

dist: 0.89
pygame.sprite.Sprite.__init__(self)

dist: 0.89
return pd.DataFrame(self.values.copy(), self.index.copy(), self.columns.copy())

dist: 0.89
def __init__(self):
    self._data = {}

